<a href="https://colab.research.google.com/github/antonimmo/ebc-jupyternotebooks/blob/master/Spectra_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --upgrade netcdf4 geopandas folium

In [0]:
# Monta la nube de GoogleDrive en el sistema "local", e importa algunas librerías necesarias
#from importlib import reload
from google.colab import drive
drive.mount('/gdrive')

In [0]:
prnt = '/gdrive/My Drive/Tesis/spectral_analysis/map_data'
#tools_fn = 'spectral_analysis_tools.py'
#from shutil import copyfile

In [0]:
import folium
from folium.plugins import MousePosition
import json
import pandas as pd
import geopandas as gpd
import branca

In [102]:
season = 'JFM' # ['ASO','JFM']
var = 'DIV'    # ['DIV','KE','RV','SSH']
prop = 'rms' # ['avg','rms','stdev']
data = pd.read_csv('{}/{}_{}_{}.csv'.format(prnt,var,season,prop))
data.head()

,Id,rms
0,603,2.365768e-09
1,872,7.193475e-09
2,584,1.653087e-09
3,295,2.202853e-09
4,532,2.585782e-09


In [103]:
#with open('{}/{}_{}_geo.json'.format(prnt,var,season),'r') as json_file:
#    geo_data = json.load(json_file)
#print(geo_data)

geodata = gpd.read_file('{}/{}_{}_geo.json'.format(prnt,var,season),driver='GeoJSON')
geodata = geodata.set_index('s_id',drop=False).join(data.set_index('Id'))
#geodata.sort_values(by=['lat','lon']).head()
min_v, max_v = geodata[prop].quantile([0.05,0.95])#.apply(lambda x: round(x, 2))
#print(min_v,max_v)
#print(geodata[prop].quantile([0.2,0.4,0.6,0.8]))
geodata[prop+'_str']=geodata[prop].apply(lambda x: "{:0.3g}".format(x)) # Formatting - tooltip
geodata.head()

,season,lat,var,lon,s_id,geometry,rms,rms_str
s_id,,,,,,,,
603,JFM,-11.032,DIV,67.0,603,"POLYGON ((64 -13.75924999999998, 64 -8.3047500...",2.365768e-09,2.37e-09
872,JFM,52.003,DIV,-23.0,872,"POLYGON ((-26 50.26125, -26 53.74475, -20 53.7...",7.193475e-09,7.19e-09
584,JFM,-16.398,DIV,91.0,584,"POLYGON ((88 -19.03675000000002, 88 -13.759249...",1.653087e-09,1.65e-09
295,JFM,-52.003,DIV,139.0,295,"POLYGON ((136 -53.74475, 136 -50.26125, 142 -5...",2.202853e-09,2.2e-09
532,JFM,-26.641,DIV,1.0,532,"POLYGON ((-2 -29.09675000000001, -2 -24.185249...",2.585782e-09,2.59e-09


In [104]:
cmap = branca.colormap.LinearColormap(
    colors=['#fcfbfd','#efedf5','#dadaeb','#bcbddc','#9e9ac8','#807dba','#6a51a3','#54278f','#3f007d'],
    #index=geodata[prop].quantile([0.2,0.4,0.6,0.8]),
    vmin=min_v,
    vmax=max_v,
    caption='{} ({}), season: {}'.format(var,prop.upper(),season)
)

style_fn = lambda x: {
    'fillColor': cmap(x['properties'][prop]),
    'color': 'black',
    'weight':2,
    'fillOpacity':0.2,
    'opacity':0.2
}

cmap

In [106]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
m = folium.Map(location=[0,0], zoom_start=2)

data_layer = folium.GeoJson(
    geodata,
    name=var,
    style_function=style_fn,
    tooltip=folium.GeoJsonTooltip(
        fields=['s_id','lat','lon',prop+'_str'],
        aliases=['Id','Lat','Lon','{} ({})'.format(var,prop.upper())],
        localize=True
    )
).add_to(m)

#folium.Choropleth(
#    #geo_data='{}/{}_{}_geo.json'.format(prnt,var,season),
#    geo_data=geo_data,
#    name='choropleth',
#    data=data,
#    columns=['Id','avg'],
#    key_on='feature.id',
#    fill_color='YlGn',
#    fill_opacity=0.2,
#    line_opacity=0.2,
#    legend_name='{} avg'.format(var),
#    control_scale=True
#).add_to(m)


m.add_child(folium.TileLayer('Stamen Terrain'))
m.add_child(folium.TileLayer('OpenStreetMap'))
m.add_child(folium.TileLayer('Stamen Toner'))
m.add_child(folium.LatLngPopup())
m.add_child(MousePosition())
m.add_child(cmap)
m.add_child(folium.LayerControl())

m